In [ ]:
import os
import xml.etree.ElementTree as ET
import pandas as pd

# Configuration
folder_path = r"C:\Users\bobbi\Documents\YEAR 3\sumo3\results\C6E30LEQ752"  # Adjusted folder path

output_csv = "AM752.csv"

def process_file(file_path):
    """
    Parse an XML file and compute average values for each metric of each lane.
    """
    lane_data = {}
    
    # Parse XML
    tree = ET.parse(file_path)
    root = tree.getroot()
    
    for data in root.findall('data'):
        lanes = data.find('lanes')
        if lanes is not None:
            for lane in lanes.findall('lane'):
                lane_id = lane.get('id')
                
                # Collect all metrics dynamically
                for key, value in lane.attrib.items():
                    if key != 'id':  # Skip the 'id' attribute
                        if lane_id not in lane_data:
                            lane_data[lane_id] = {}
                        if key not in lane_data[lane_id]:
                            lane_data[lane_id][key] = []
                        lane_data[lane_id][key].append(float(value))
    
    # Calculate averages for all metrics
    avg_metrics = {}
    for lane_id, metrics in lane_data.items():
        for metric, values in metrics.items():
            avg_key = f"{lane_id}_{metric}_avg"
            avg_metrics[avg_key] = sum(values) / len(values) if values else 0
    
    return avg_metrics

def aggregate_files(folder_path):
    """
    Aggregate data from all files into a DataFrame, processing only files that start with "queue".
    """
    rows = []
    all_columns = set()
    
    for file_name in os.listdir(folder_path):
        if file_name.startswith('queue') and file_name.endswith('.xml'):
            file_path = os.path.join(folder_path, file_name)
            avg_metrics = process_file(file_path)
            
            # Add to the result row
            row = {"file_name": file_name}
            row.update(avg_metrics)
            rows.append(row)
            
            # Collect all column names
            all_columns.update(avg_metrics.keys())
    
    # Create a DataFrame with all metrics as columns
    df = pd.DataFrame(rows)
    df = df.set_index("file_name")  # Use file names as the index
    df = df.reindex(columns=sorted(all_columns))  # Ensure all metrics are included as columns
    df = df.reset_index()  # Reset index to include file names in the output
    return df

# Process the folder and save the CSV
df = aggregate_files(folder_path)
df.to_csv(output_csv, index=False)
print("Aggregated metrics saved to:", output_csv)


Aggregated metrics saved to: AMMIS2.csv
